In [1]:
# Librerías
import pandas as pd
import numpy as np
from pathlib import Path
import os

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

In [2]:
sns.set_theme(style="darkgrid")

In [3]:
# Variables
src_path = os.getcwd()
data_path = "/data/raw/"
data_file = "merged_data.csv"

file_path= Path(os.path.dirname(src_path)+data_path+data_file)

In [4]:
df_coffee = pd.read_csv(file_path)

In [5]:
df_coffee.shape

(1339, 44)

In [6]:
df_coffee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1339 entries, 0 to 1338
Data columns (total 44 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             1339 non-null   int64  
 1   Species                1339 non-null   object 
 2   Owner                  1332 non-null   object 
 3   Country.of.Origin      1338 non-null   object 
 4   Farm.Name              980 non-null    object 
 5   Lot.Number             276 non-null    object 
 6   Mill                   1021 non-null   object 
 7   ICO.Number             1180 non-null   object 
 8   Company                1130 non-null   object 
 9   Altitude               1113 non-null   object 
 10  Region                 1280 non-null   object 
 11  Producer               1107 non-null   object 
 12  Number.of.Bags         1339 non-null   int64  
 13  Bag.Weight             1339 non-null   object 
 14  In.Country.Partner     1339 non-null   object 
 15  Harv

<h2 style="text-align: center;">ANALIZANDO EL DATASET</h2>

El dataset consta de 1339 filas por 44 columnas y contiene la información relativa a diferentes variedades de café de tipo Arabica y Robusta calificadas por personal del CQI (Coffee Quality Institute).

La información se divide en:

Calidad de café (Cupping Score)
- Aroma
- Flavor
- Aftertaste
- Acidity
- Body
- Balance
- Uniformity
- Clean Cup
- Sweetness


Semilla (Bean)
- Species (arabica / robusta)
- Variety
- Processing Method
- Moisture
- Color
- Category.One.Defects
- Category.Two.Defects
- Quakers
- Expiration


Plantación (Farm)
- Country.of.Origin
- Harvest.Year
- Farm.Name
- Mill
- Owner
- Company
- Altitude + unit_of_measurement
- Region



La columna "Total Cup Points" será nuestro medidor de calidad, siendo mejor el café cuanto más alta sea esta puntuación.

Realizaremos un análisis univariante y multivariante de las información de la calidad y de la semilla junto con el país de origen, región, año de cosecha y altitud de la plantación con el objetivo final de encontrar un algoritmo que permita predecir qué café tendrá una mejor puntuación en función de todas sus características.

Para la información de la altitud, nos quedaremos con la columna "Altitude" y "altitude_mean_meters" que representa la media entre el valor máximo y mínimo de la altitud a la que se cultiva esa variedad de café en esa plantación. Además, necesitamos la columna "unit_of_measurement" para saber si la medida está en metros o pies.

<h2 style="text-align: center;">LIMPIEZA DEL DATASET</h2>

Borramos las columnas que no nos serán necesarias para la realización del análisis.

In [7]:
df = df_coffee.drop(columns=["Unnamed: 0", "Lot.Number", "ICO.Number", "Producer", "Number.of.Bags",
                             "Bag.Weight", "In.Country.Partner", "Grading.Date", "Cupper.Points",
                             "Certification.Body", "Certification.Address", "Certification.Contact",
                             "altitude_low_meters", "altitude_high_meters", "Owner.1", "Farm.Name",
                             "Mill", "Company", "Owner"])

<h3 style="text-align: center;">Chequeo de duplicados</h3>

In [8]:
# Cantidad de duplicados
df.duplicated().sum()

0

No hay registros duplicados.

<h3 style="text-align: center;">Chequeo de valores nulos</h3>

In [9]:
# Null / Missing values
df[df.columns[df.isnull().any()]].isnull().sum().sort_values(ascending=False)

Color                   270
altitude_mean_meters    230
Altitude                226
Variety                 226
Processing.Method       170
Region                   59
Harvest.Year             47
Country.of.Origin         1
Quakers                   1
dtype: int64

Hay bastante cantidad de valores nulos. Vamos a tratar de reducirlos todo lo posible.

<h4 style="text-align: center;"><ins>1 - Nulos en Region</ins></h4>

Saber la región en la que se cultiva un café puede ser importante para sus características.

Comprobamos si podemos inferir a qué región pertenece un café en función de su país de origen y la altitud media a la que se encuentra la plantación. La idea es que si uno de los registros nulos comparte altura y pais de origen con otro que que tenga un valor, supondremos que se trata de la misma región.

In [10]:
df.loc[(df["Region"].isnull()) & (df["Country.of.Origin"].notnull()), ["Country.of.Origin", "Region", 'altitude_mean_meters']]

,Country.of.Origin,Region,altitude_mean_meters
2,Guatemala,NaN,1700.0
5,Brazil,NaN,NaN
6,Peru,NaN,NaN
30,Nicaragua,NaN,NaN
31,"Tanzania, United Republic Of",NaN,NaN
100,Kenya,NaN,1.0
114,Kenya,NaN,NaN
134,Costa Rica,NaN,NaN
152,Kenya,NaN,NaN
223,Guatemala,NaN,NaN


La práctica totalidad de los registros con valor nulo en el campo de Región también presenta valor nulo para su altura media, con lo que es imposible imputar valores y por tanto decidimos eliminar estos registros.

In [11]:
df = df[df["Region"].notna()]

<h4 style="text-align: center;"><ins>2 - Nulos en altitud</ins></h4>


Uno de los valores que hemos considerado importante en el análisis es la altitud a la que se cultiva el café, por lo que es el primer dato que vamos a analizar 

1 - Comprobamos los valores de la columna "Altitud" que no son nulos junto con los de altitud media para tratar de imputar algún valor a esta última columna.

In [12]:
df.loc[(df["altitude_mean_meters"].isnull()) & df["Altitude"].notnull(), ["Altitude", "altitude_mean_meters"]]

,Altitude,altitude_mean_meters
169,mmm,NaN
170,test,NaN
687,huanuco,NaN
986,0,NaN


Con esos valores no podemos realizar ningún cálculo.

Antes de realizar las imputaciones debemos tener en cuenta la unidad de medida en que está la columna de altitud y asegurarnos que todos los valores están en metros.

In [13]:
df["altitude_recalc"] = df.apply(lambda x: x["altitude_mean_meters"] if x["unit_of_measurement"]=='m' else x["altitude_mean_meters"]*0.3048, axis=1)

Estudiamos los valores estadísticos de la columna, pues si vamos a realizar imputaciones a la media o a la moda debemos asegurarnos de que no existan valores fuera de rango que nos provoquen errores.

In [ ]:
df["altitude_recalc"].describe()

2 - Imputaremos los valores de altitud correspondientes a la media de la región a la que pertenecen.

In [ ]:
df["altitude_recalc"].fillna(df.groupby(["Region"])["altitude_recalc"].transform("mean"), inplace=True)
print("Quedan {} valores nulos en la columna.". format(df["altitude_recalc"].isnull().sum()))

3 - Dado que aún quedan valores nulos, vamos a repetir el proceso pero imputando la moda de la altura con respecto al país. Decidimos no usar la media porque puede ocurrir que en un mismo país haya grandes diferencias en la altura en que sitúan las plantaciones.

In [ ]:
df["altitude_recalc"].fillna(df.groupby(['Country.of.Origin'])['altitude_recalc'].transform(lambda x: x.mode()[0]), inplace=True)
print("Quedan {} valores nulos en la columna.". format(df["altitude_recalc"].isnull().sum()))

Hemos quitado todos los valores nulos de la altitud media, por lo que la columna Altitude se hace innecesaria en el conjunto de datos y procedemos a eliminarla.

In [ ]:
df = df.drop(columns=["Altitude"])

<h4 style="text-align: center;"><ins>3 - Nulos en variedad</ins></h4>

Antes de estudiar los registros nulos comprobamos los diferentes tipos de variedad que hay en la tabla

In [ ]:
df["Variety"].unique()

Dado que ya existe un valor genérico establecido como "Other" decidimos imputarles a todos los nulos ese valor genérico.

In [ ]:
df["Variety"].fillna("Other", inplace = True)
print("Quedan {} valores nulos en la columna.". format(df["Variety"].isnull().sum()))

<h4 style="text-align: center;"><ins>4 - Nulos en Color</ins></h3>

Antes de estudiar los valores nulos, comprobamos los diferentes colores presentes en el dataset

In [ ]:
df["Color"].unique()

El color de las semillas de café, antes del tostado, suele depender de la región en la que se hayan cultivado, por lo que en una primera fase imputaremos a los valores nulos el color más habitual en la región en la que son cultivados.

In [ ]:
df["Color"].fillna(df.groupby(["Region"])["Color"].transform(lambda x: x.mode(dropna=False)[0]), inplace = True)
print("Quedan {} valores nulos en la columna.". format(df["Color"].isnull().sum()))

La imputación al color habitual de la región no ha sido de gran ayuda, por lo que repetiremos el proceso pero con la moda del color del país.

In [ ]:
df["Color"].fillna(df.groupby(['Country.of.Origin'])['Color'].transform(lambda x: x.mode(dropna=False)[0]), inplace = True)
print("Quedan {} valores nulos en la columna.". format(df["Color"].isnull().sum()))

Dado que aún nos restan valores nulos y que existe una categoría que es "None" imputaremos a ese valor todos los nulos restantes

In [ ]:
df["Color"].fillna("None", inplace=True)
print("Quedan {} valores nulos en la columna.". format(df["Color"].isnull().sum()))

<h4 style="text-align: center;"><ins>5 - Nulos en Método de procesado</ins></h3>

Como anteriormente, vamos a comprobar los tipos de métodos de procesado existentes en el dataframe.

In [ ]:
df["Processing.Method"].unique()

Dado que el método de procesado es difícil de imputar pues no tiene por qué seguir una lógica en función de la región o el país, vamos a imputar a todos los nulos el valor genérico "Other" que ya existe.

In [ ]:
df["Processing.Method"].fillna("Other", inplace=True)
print("Quedan {} valores nulos en la columna.". format(df["Processing.Method"].isnull().sum()))

<h4 style="text-align: center;"><ins>6 - Nulos en Año de cosecha</ins></h3>

In [ ]:
df["Harvest.Year"].unique()

Viendo la cantidad de variabilidad que hay en la columna, decidimos extraer la última aparición de una cifra de 4 dígitos y tomarlos como el año de la cosecha. Almacenaremos el resultado en una nueva columna.

In [ ]:
df["HarvestYear_Calc"] = df["Harvest.Year"].str.extract('(\d{4})(?!.*\d)')

Examinamos de nuevo los valores obtenidos tras la imputación.

In [ ]:
df["HarvestYear_Calc"].unique()

Nos quedan valores nulos, a los que imputaremos el valor genérico "unknown", pues para tratarlos deberíamos ir uno por uno.

In [ ]:
df["HarvestYear_Calc"].fillna("UNKNOWN", inplace=True)
print("Quedan {} valores nulos en la columna.". format(df["HarvestYear_Calc"].isnull().sum()))

Ya no es necesaria la columna original del año de cosecha, por lo que podemos eliminarla

In [ ]:
df.drop(columns=["Harvest.Year"], inplace=True)

<h4 style="text-align: center;"><ins>7 - Nulos en Quakers</ins></h3>

Los "Quakers" son aquéllas semillas de café que, por no estar maduras y por tanto no tener suficientes azúcares, no se caramelizan con el tostado y si no se retiran pueden provocar un empeoramiento de la calidad del café.

Esta columna indica el número de estas semillas encontradas en el saco de café.

In [ ]:
df[df["Quakers"].isnull()]

Como sólo tenemos un caso con un valor nulo y no podemos imputar ningún valor pues es una característica independiente a la región o al país, lo eliminaremos sin que afecte al desarrollo del análisis.

In [ ]:
df = df[df["Quakers"].notna()]

---

Volvemos a comprobar los valores nulos, para asegurar que no nos queda ninguno.

In [ ]:
# Null / Missing values
df[df.columns[df.isnull().any()]].isnull().sum().sort_values(ascending=False)

<h2 style="text-align: center;">ANÁLISIS DE VARIABLES</h2>

<h4 style="text-align: center;"><ins>1 - País de Origen</ins></h4>

Examinamos los valores únicos de la columna, por si fuera necesario realizar alguna codificación o traducción en sus valores.

In [ ]:
df["Country.of.Origin"].unique()

Los valores son relativamente pocos y no es necesario realizar ningún cambio.

<h4 style="text-align: center;"><ins>2 - Especie</ins></h4>

Examinanos las especies presentes en el dataset.

In [ ]:
df["Species"].unique()

Solamente hay dos especies y ningún registro carece de esta información.

<h4 style="text-align: center;"><ins>3 - Variedad</ins></h4>

Examinamos los tipos de variedad presentes en el conjunto de datos.

In [ ]:
df["Variety"].unique()

Aunque son bastantes, no existe ningún valor extraño y todo aquéllo que no está definido tiene el valor genérico "Other"

<h4 style="text-align: center;"><ins>4 - Altitud</ins></h4>

Analizamos los valores de la columna relativa a la altitud media del cultivo. Dado que es un valor numérico, además del número de valores que presenta realizaremos un análisis de sus valores estadísticos.

In [ ]:
df["altitude_recalc"].describe()

Tener tanta variedad de alturas medias va a suponer un problema a la hora de analizar los datos y en el momento de las predicciones, por lo que se hace necesario realizar agrupamientos de valores a fin de simplificar esta categoría.

Además, podemos observar un valor máximo demasiado elevado que debemos tratar.

Para localizar los valores outliers en la columna, usaremos el valor de su zscore o número de desviaciones estándar de la media. Aquéllos con un valor superior a 3 serán considerados outliers.

In [ ]:
df["altitude_recalc_zscore"]=np.abs(stats.zscore(df["altitude_recalc"]))
df.loc[df["altitude_recalc_zscore"]>3, ["altitude_recalc", "altitude_recalc_zscore"]]

Dado que sólo hemos encontrado tres casos, decidimos eliminarlos.

In [ ]:
df = df[df["altitude_recalc_zscore"]<3]

In [ ]:
df.loc[df["altitude_recalc"]>4000]

Seguimos teniendo demasiados valores, por

---

<h4 style="text-align: center;"><ins>1 - País de Origen</ins></h4>

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
sns.countplot(data=df, y="Country.of.Origin", order= df["Country.of.Origin"].value_counts().index, ax=ax)
plt.show()

Los principales países productores de café presentes en el dataset son centroamericanos, con algunos africanos en posiciones centrales y los asiáticos con muy poca representación.

<h4 style="text-align: center;"><ins>2 - Especie</ins></h4>

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
sns.countplot(data=df, y="Species", order= df["Species"].value_counts().index, ax=ax)
plt.show()

El dataset está claramente poblado por más cantidad de registros de la especie "Arabica" que de la especie "Robusta".

Esto debe ser tenido en cuenta en el momento de la realización de predicciones, pues un desequilibrio tan grande afectará negativamente y ha de ser evitado.

<h4 style="text-align: center;"><ins>3 - Variedad</ins></h4>

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
sns.countplot(data=df, y="Variety", order= df["Variety"].value_counts().index, ax=ax)
plt.show()

La presencia de tantos registros sin un valor específico de la variedad de café cultivada nos afectará negativamente a la hora de predecir qué variedad es mejor, por lo que lo mejor sería no tener en cuenta este atributo en su momento, aunque vamos a mantenerlo durante el análisis.

<h4 style="text-align: center;"><ins>4 - Altitud</ins></h4>

In [ ]:
df["altitude_mean_meters"].agg(['min', 'max'])

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
sns.countplot(data=df, y="altitude_mean_meters", order= df["altitude_mean_meters"].value_counts().index, ax=ax)
plt.show()

In [ ]:
<h4 style="text-align: center;"><ins>2 - Especie</ins></h4>

df.columns

#### Checking values for VARIETY

In [ ]:
df["Variety"].unique()

There are multiple and different values for unknown varieties or missing values. We will standarize it and assign the value "UNKNOWN" for all of them.

Also, we will replace the '+' symbols as well as 'and' or 'y' words with commas to correctly separate the coffee varieties.

To finish, we will write all the words in capital letters.

In [ ]:
df["Variety"][(df["Variety"]=='unknown') | (df["Variety"]=='unknow')] = "UNKNOWN"
df.loc[df["Variety"].str.contains(' \+ '), 'Variety'] = df["Variety"].str.replace(' + ',',')
df.loc[df["Variety"].str.contains('\+'), 'Variety'] = df["Variety"].str.replace('+',',')
df.loc[df["Variety"].str.contains(' and '), 'Variety'] = df["Variety"].str.replace(' and ',',')
df.loc[df["Variety"].str.contains(' Y '), 'Variety'] = df["Variety"].str.replace(' Y ',',')
df.loc[df["Variety"].str.contains(' & '), 'Variety'] = df["Variety"].str.replace(' & ',',')
df["Variety"] = df["Variety"].str.upper()


#### Checking values for PROCESSING METHOD

In [ ]:
df["Processing Method"].unique()

We replace the method in spanish for the translated one, which is "Semi Washed" and set all values to capital letters

In [ ]:
df["Processing Method"][(df["Processing Method"]=='SEMI-LAVADO')] = 'Semi Washed'
df["Processing Method"] = df["Processing Method"].str.upper()